## Dataset notes
Training will fail with samples that are too short (<1 second).

## Download necessary files and set base config
Specify the experiment name in `EXPERIMENT_NAME` and download the dataset and OOD text list. This will also configure some default settings in `Configs/config.yml`.

`DATASET_URL` should point to a google drive hosted .zip containing:

* train_list.txt
* val_list.txt
* *.wav

`OOD_URL` should point to an OOD text list (the corresponding audio files are not needed; this is just used to test the TTS model on out-of-dataset text)

`PRETRAINED`, if populated, should point to a direct download for a first-stage checkpoint or pretrained second-stage model (depending on what stage of training you plan to do.)

It is recommended to use huggingface if your home internet bandwidth is not fantastic.
If it is great, you can use `vastai copy` to copy files to the desired locations. These are the files you need to provide:
* `/root/StyleTTS2/Data/train_list.txt`
* `/root/StyleTTS2/Data/val_list.txt`
* `/root/StyleTTS2/{EXPERIMENT_NAME}_data/*.wav`
* `/root/StyleTTS2/Models/{EXPERIMENT_NAME}/pretrained.pth` (if using a pretrained model)

In [4]:
EXPERIMENT_NAME = "Twilight0"
DATASET_URL = "https://huggingface.co/datasets/therealvul/StyleTTS2MLP/resolve/main/StyleTTS2Twilight.zip?download=true"
OOD_URL = "https://gist.githubusercontent.com/effusiveperiscope/f3a6d48ad3463b63d1cb9a53bfab9dd9/raw/480f168c0f4cdc96c8863ac09c84978b364ee6a1/gistfile1.txt"
PRETRAINED_URL = "https://huggingface.co/therealvul/StyleTTS2/resolve/main/Unfinished/epoch_1st_00051.pth?download=true"



In [18]:
import ffmpeg
import os
import gdown
import urllib
from tqdm import tqdm
from pathlib import Path

# Download and extract dataset and data labels
styletts_basedir = "/root/StyleTTS2" 
dataset_file = os.path.join(styletts_basedir, f"dataset_{EXPERIMENT_NAME}.zip")
dataset_target_dir = os.path.join(styletts_basedir, f"{EXPERIMENT_NAME}_data")
model_dir = os.path.join(styletts_basedir, "Models", EXPERIMENT_NAME)

def download_file(url, dest):
    print(f"Downloading {url} to {dest}")
    with urllib.request.urlopen(url) as r, open(dest, 'wb') as out_file:
        total_size = int(r.info().get('Content-Length', 0))
        block_size = 1024
        with tqdm(total=total_size, unit='B', unit_scale=True) as pbar:
            while True:
                data = r.read(block_size)
                if not data:
                    break
                out_file.write(data)
                pbar.update(len(data))

print("Downloading dataset")
if not os.path.exists(dataset_file):
    if "drive.google.com" in DATASET_URL:
        gdown.download(DATASET_URL, output=dataset_file, quiet=False)
    else:
        download_file(DATASET_URL, dataset_file)
print("Downloaded dataset")

assert(os.path.exists(dataset_file))

if not os.path.exists(dataset_target_dir):
    os.makedirs(dataset_target_dir, exist_ok=True)

import zipfile
with zipfile.ZipFile(dataset_file, 'r') as f:
    files_count = len(f.namelist())
    with tqdm(total=files_count, desc=f"Extracting data",
        unit="file") as pbar:
        for info in f.namelist():
            if info.endswith('.txt'):
                unzip_file_path = os.path.join(styletts_basedir, 'Data')
                f.extract(info, unzip_file_path) # Not sure if this can overwrite data?
            elif info.endswith('.wav'):
                unzip_file_path = dataset_target_dir
                if os.path.exists(os.path.join(unzip_file_path, info)):
                    continue
                f.extract(info, unzip_file_path)
            pbar.update(1)

# Download OOD text
ood_out_path = os.path.join(styletts_basedir, 'Data', 'OOD_texts.txt')
if not os.path.exists(ood_out_path):
    print("Downloading OOD")
    download_file(OOD_URL, ood_out_path)
    print("Downloaded OOD")

# Download pretrained model
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

if PRETRAINED_URL is not None and not os.path.exists(os.path.join(model_dir, 'pretrained.pth')):
    print("Downloading pretrained model")
    download_file(PRETRAINED_URL, os.path.join(model_dir, 'pretrained.pth'))
    print("Downloaded pretrained model")

# Setup base config
import yaml
config_path = os.path.join(styletts_basedir, 'Configs', 'config.yml')
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
    config['log_dir'] = f'Models/{EXPERIMENT_NAME}'
    if PRETRAINED_URL is not None:
        config['pretrained_model'] = os.path.join(model_dir, "pretrained.pth")
    config['data_params']['root_path'] = dataset_target_dir

with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

Downloaded dataset


Extracting data:   0%|          | 2/6199 [00:00<05:22, 19.23file/s]


100%|██████████| 58.4k/58.4k [00:00<00:00, 8.07MB/s]

Downloaded OOD


## First stage (acoustic reconstruction training)
This is only for training a model from scratch; you do not need to run this step if you only plan to finetune the diffusion (second stage) model.

Configure the settings in the cell before running.

* `TOTAL_EPOCHS` - total epochs used for first stage training. The paper used `100` epochs for LJSpeech (single speaker, 24hrs), `50` epochs for VCTK (109 speakers, 44hrs), and `30` epochs for LibriTTS (1151 speakers, 245hrs).
* `TMA_START_POINT` - epochs before which transferable monotonic alignment is trained. Epochs in the TMA training regime are typically 20x longer than epochs before--so this should be expected to take up a large portion of training time. [The author used TMA_epoch = 5 for LibriTTS](https://github.com/yl4579/StyleTTS2/issues/18). I found `50` to be a reasonable value for a 5-hour dataset, so this can probably be lowered for larger datasets.
* `LOAD_ONLY_PARAMS` - Whether only parameters are loaded (vs. loading optimizer state and epoch information). If you are continuing the training process of an existing first-stage checkpoint, this should be `False`.
* `USE_CHECKPOINT` - Manually specify a checkpoint path to use. This may be useful, for example, if you needed to interrupt training to modify training parameters.
* `SAVE_FREQ` - Determines how often epochs are saved. Be mindful of your allocated disk limits. Because TMA epochs take so long to complete, you may want to consider lowering this down to 1 during TMA training.
* `BATCH_SIZE` - Higher values will increase training speed but require more VRAM. A batch size of 2 will fit on a 16GB GPU during both training stages. 

In [21]:
TOTAL_EPOCHS = 100 
TMA_START_POINT = 50 
LOAD_ONLY_PARAMS = True
USE_CHECKPOINT = None
SAVE_FREQ = 2
BATCH_SIZE = 4

styletts_basedir = styletts_basedir or "/root/StyleTTS2"
config_path = config_path or os.path.join(styletts_basedir, 'Configs', 'config.yml')
pretrained_model = os.path.join(styletts_basedir, "Models", EXPERIMENT_NAME, "pretrained.pth")

import os
os.chdir(styletts_basedir)

import yaml
with open(config_path) as f:
    config = yaml.safe_load(f)
    config['epochs_1st'] = TOTAL_EPOCHS
    config['loss_params']['TMA_epoch'] = TMA_START_POINT
    config['load_only_params'] = LOAD_ONLY_PARAMS
    if (USE_CHECKPOINT is not None):
        config['pretrained_model'] = USE_CHECKPOINT
    elif os.path.exists(pretrained_model):
        config['pretrained_model'] = pretrained_model
    else:
        config['pretrained_model'] = ""
    config['save_freq'] = SAVE_FREQ
    config['batch_size'] = BATCH_SIZE
    config['second_stage_load_pretrained'] = False

with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

!accelerate launch --mixed_precision=fp16 train_first.py --config_path ./Configs/config.yml

## Second stage (diffusion training/finetuning)
[This discussion page](https://github.com/yl4579/StyleTTS2/discussions/81)
contains some useful information on finetuning. Some gotchas:
* It is basically impossible to finetune on a 16GB GPU--you need 24GB minimum. The original models were trained on 4xA100 (e.g. 160 GB VRAM).
* Epochs are zero-indexed in the second stage. This includes `DIFF_BEGIN_EPOCH` and `JOINT_EPOCH`.

Configure the settings in the below cell before running.
* `IS_FIRST_STAGE` - **!!! Set this if the checkpoint provided is a first stage checkpoint !!!**.
* `MAX_LEN` - the maximum length of training samples, in frames (.0125 second/frame; 100 frames = 1.25 seconds, 800 frames = 10 seconds). This will affect VRAM usage and the ability of the model to coherently synthesize longer samples. A `MAX_LEN` below 175 is not recommended. Do not reduce `MAX_LEN` below 100.
* `TOTAL_EPOCHS` - total epochs used for second stage training. The paper used 60 epochs on LJSpeech, 40 epochs on VCTK, and 25 epochs on LibriTTS.
* `DIFF_BEGIN_EPOCH` - The epoch at which style diffusion training begins. This can be as low as `1` (i.e. second epoch) for large datasets. If you set this above `TOTAL_EPOCHS`, you can disable style vector diffusion, lowering VRAM usage at the cost of output quality. 
* `JOINT_BEGIN_EPOCH` - The epoch at which SLM adversarial training begins. This cannot be set lower than `DIFF_BEGIN_EPOCH` or an error will occur (SLM adversarial cannot be run without diffusion). SLM adversarial training will increase VRAM usage. A model can be produced without adversarial training, at the expense of output quality.
* `BATCH_SIZE` - Higher values will increase training speed but require more VRAM. **Will break if lowered below (NUM_GPUS) * 2.**
* `USE_CHECKPOINT` - Manually specify a checkpoint path to use. *This may be useful for the technique described in the above discussion where you can produce a diffusion-trained-only checkpoint at a higher batch size, then shift to a lower batch size for SLM training.*
* `SAVE_FREQ` - Determines how often epochs are saved. Be mindful of your allocated disk limits. Because TMA epochs take so long to complete, you may want to consider lowering this down to 1 during TMA training.
* `LOAD_ONLY_PARAMS` - Whether only parameters are loaded (vs. loading optimizer state and epoch information). If you are starting a fresh finetune from a pretrained **2nd stage** checkpoint, set to `True`. If you are starting a fresh finetune from a pretrained 1st stage check point, or if you are continuing the training process of a second-stage checkpoint, set to `False`.
* `DISABLE_AUTORESUME_BEHAVIOR` - **For my ease of use, this cell will automatically load the most recent 2nd stage checkpoint if found in the model folder.** This helps with stopping and starting training at different batch sizes for diffusion or slmadv. Setting this to `True` will disable this behavior and allow you to use

In [ ]:
IS_FIRST_STAGE = False
MAX_LEN = 175
TOTAL_EPOCHS = 5
DIFF_BEGIN_EPOCH = 1
JOINT_BEGIN_EPOCH = 5
BATCH_SIZE = 4
SEGMENTED_BATCH_SIZE = [16, 8, 6] # works on 45GB
USE_CHECKPOINT = None
SAVE_FREQ = 2
LOAD_ONLY_PARAMS = False and not IS_FIRST_STAGE
DISABLE_AUTORESUME_BEHAVIOR = False

styletts_basedir = styletts_basedir or "/root/StyleTTS2"
config_path = config_path or os.path.join(styletts_basedir, 'Configs', 'config.yml')
pretrained_model = os.path.join(styletts_basedir, "Models", EXPERIMENT_NAME, "pretrained.pth")

import re
if (not DISABLE_AUTORESUME_BEHAVIOR):
    fs = os.listdir(f'/root/StyleTTS2/Models/{EXPERIMENT_NAME}')
    pretrained_matches = sorted(
        filter(lambda x: x.startswith("epoch_2nd_"), fs),
        key=lambda x: int(re.search(r'epoch_2nd_(\d+).pth', x).group(1)))
    if len(pretrained_matches):
        found_model = pretrained_matches[-1]
        print(f"========== USING FOUND 2ND STAGE CKPT: {found_model} ====================")
        pretrained_model = found_model
        USE_CHECKPOINT = found_model

import os
os.chdir(styletts_basedir)

import yaml
with open(config_path) as f:
    config = yaml.safe_load(f)
    config['max_len'] = MAX_LEN
    config['epochs_2nd'] = TOTAL_EPOCHS
    config['loss_params']['diff_epoch'] = DIFF_BEGIN_EPOCH
    config['loss_params']['joint_epoch'] = JOINT_BEGIN_EPOCH
    config['batch_size'] = BATCH_SIZE
    config['segmented_batch_size'] = SEGMENTED_BATCH_SIZE
    config['second_stage_load_pretrained'] = not IS_FIRST_STAGE
    if (USE_CHECKPOINT is not None):
        # If IS_FIRST_STAGE is false this is ignored anyways
        config['first_stage_path'] = pretrained_model 
        config['pretrained_model'] = USE_CHECKPOINT
    elif os.path.exists(pretrained_model):
        config['first_stage_path'] = pretrained_model
        config['pretrained_model'] = pretrained_model
    else:
        config['first_stage_path'] = ""
        config['pretrained_model'] = ""
    config['save_freq'] = SAVE_FREQ
    config['load_only_params'] = LOAD_ONLY_PARAMS

with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

#!accelerate launch --mixed_precision=fp16 train_second.py --config_path ./Configs/config.yml
!accelerate launch --mixed_precision=fp16 train_second_segmented.py --config_path ./Configs/config.yml

## Run tensorboard logger

In [ ]:
%load_ext tensorboard
%tensorboard --logdir os.path.join(styletts_basedir, "Models", EXPERIMENT_NAME, "tensorboard")